In [1]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
from numpy.linalg import matrix_power

In [2]:
Target_state = '000000'

In [3]:
# First we note the length of N.
L = len(Target_state)

N = 2**L

# Then an identity matrix is created with the size 2**N with signs reversed.
U_w = - np.identity(2 ** L, dtype=complex)


# Then the sign of the element corresponding to the target state is flipped. To do that we first convert the
# target state from binary to decimal number. 
Target_index = int(Target_state, 2)

## The position of the target state is set as 1.
U_w.itemset((Target_index, Target_index),1)

In [4]:
## We will first create a matrix with all elements 1. This is |psi><psi| = A(say).
A = np.ones((2**L, 2**L))

## U_s = 2\(2**N)2|psi><psi| - I
U_s = (2/(2**L))*A - np.identity(2**L)

## G is the Grover operator.
G = np.matmul(U_w, U_s)


print('The Grover operator for the target state |w > = | '+Target_state + ' > is \n\n',G.real)

The Grover operator for the target state |w > = | 000000 > is 

 [[-0.96875  0.03125  0.03125 ...  0.03125  0.03125  0.03125]
 [-0.03125  0.96875 -0.03125 ... -0.03125 -0.03125 -0.03125]
 [-0.03125 -0.03125  0.96875 ... -0.03125 -0.03125 -0.03125]
 ...
 [-0.03125 -0.03125 -0.03125 ...  0.96875 -0.03125 -0.03125]
 [-0.03125 -0.03125 -0.03125 ... -0.03125  0.96875 -0.03125]
 [-0.03125 -0.03125 -0.03125 ... -0.03125 -0.03125  0.96875]]


In [5]:
Num_iteration = int(np.pi/4 * np.sqrt(N))

The operator we will unfold into two qubit gates is $$ G = (2( |\psi><\psi|-I)O)^M $$

In [6]:
#G = matrix_power(V,Num_iteration)

The following function calculates two level unitary matrices for a matrix of size
greater than 3, returns a ordered pair of list
 of matrices U and the final product of all the U matrices.
 Following the algorithm given in Nielsen, M.A. and Chuang.

In [7]:
def ReductionN(Matrix):

    Two_Level_Universal_Gate = []
    
    # Extract the size of the input matrix.
    Matrix_dim = int(np.sqrt(Matrix.size))

    for i in range(1,Matrix_dim):

        if Matrix[i,0] == 0:

            U = np.identity(Matrix_dim, dtype = complex)

            if i == 1:
                pass
            else:
                U.itemset((0,0), np.conjugate(Matrix[0,0]))

        else:

            a, b = Matrix[0,0], Matrix[i,0]

            u00 = np.conjugate(a) / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u01 = np.conjugate(b) / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u10 = b / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u11 = - a / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    
            U = np.identity(Matrix_dim, dtype = complex)
            U.itemset((0,0), u00)
            U.itemset((0,i), u01)
            U.itemset((i,0), u10)
            U.itemset((i,i), u11)
        
        Two_Level_Universal_Gate.append(U)

        # Calculating the product of all the U matrices with G.

        Matrix = np.matmul(U, Matrix)

    return Two_Level_Universal_Gate, Matrix

Function calculates two level unitary matrices for a 3x3 matrix.
    Returns a list containing the two level matrices.
    The function will be called reduced as it put zeros in the first 
    column and the first row.In what follows, the final matrix product
    will be called reduced matrix.

In [8]:
def Reduction3(Matrix):

    Two_Level_Universal_Gate = []
    
    # Size of the matrix is 3.
    Matrix_dim = 3
 
    for i in range(1,Matrix_dim):

        '''
            If the (1,0) element is 0, then U_1 = identity matrix.
        
        '''
        
        if Matrix[i,0] == 0:

            U = np.identity(Matrix_dim, dtype = complex) # U_1 is identity.

            # For the first iteration it is identity.
            if i == 1:
                
                pass
            
            else:
                
                U.itemset((0,0), np.conjugate(Matrix[0,0]))

        else:

            a, b = Matrix[0,0], Matrix[i,0]

            u00 = np.conjugate(a) / np.sqrt(np.absolute(a)**2 \
                + np.absolute(b)**2)
            u01 = np.conjugate(b) / np.sqrt(np.absolute(a)**2 \
                + np.absolute(b)**2)
            u10 = b / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u11 = - a / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    
            U = np.identity(Matrix_dim, dtype = complex)
            U.itemset((0,0), u00)
            U.itemset((0,i), u01)
            U.itemset((i,0), u10)
            U.itemset((i,i), u11)
        
        Two_Level_Universal_Gate.append(U)

        # Calculating product of all the U matrices with G.
        Matrix = np.matmul(U, Matrix)

    # U3 is calculated sepatarely.
    Two_Level_Universal_Gate.append(np.conjugate(Matrix).transpose())

    return Two_Level_Universal_Gate

The following function returns a sub matrix with one dimension lower than the input matrix.

In [9]:
def Extract_subMatrix(Matrix):

    Matrix_size = int(np.sqrt(Matrix.size))
    subMatrix = Matrix[1:Matrix_size, 1:Matrix_size]

    return  subMatrix

The following function expands a given matrix to a required size.

In [10]:
def Expand_Matrix(Matrix, Required_Size):

    # Intital size of the input matrix.
    Initial_Size = int(np.sqrt(Matrix.size))

    # Creating an initial identity matrix.
    M = np.identity(Required_Size, dtype = complex)

    # Size difference between the two matrics.
    Sz = Required_Size - Initial_Size

    # The elements from the old matrix is added to M, other row and column are part of
    # identity matrix, this gurantees that the matrix is a two level unitary matrix.

    for i in range(Sz, Required_Size):
        
        for j in range(Sz, Required_Size):
            
            M.itemset((i,j), Matrix[i-Sz,j-Sz])

    return M

In [11]:
N

64

This loop will calculate the two level unitary matrices using the functions described above.

In [12]:
def Reduce(Matrix):
    # This list will contain all the two level unitary gates.
    Unitary_gates = []

    Last_Reduced_Matrix = []

    for i in range(N, 2, -1): # Loop from N to 3.
    
    
        # 3x3 matrices are treated separetely with CReduction3(Matrix).
        if i == 3:
        
            M = Last_Reduced_Matrix[-1]

            for g in Reduction3(M):
            
                Unitary_gates.append(g)

        # First iteration, start with original matrix G.
        elif i == N:

            M = Matrix
            
            for g in ReductionN(M)[0]: # List of two level gates.
            
                Unitary_gates.append(g)

        
            # The reduced submatrix is put into the list to use in the next loop.
            Last_Reduced_Matrix.append(Extract_subMatrix(ReductionN(M)[1]))
            #print(Last_Reduced_Matrix)
        
        else:
        
            # Any other dimension is reduced with CReductionN(Matrix).
            M = Last_Reduced_Matrix[-1]

            for g in ReductionN(M)[0]:
            
                Unitary_gates.append(g)   

            # The reduced submatrix is put into the list to use in the next loop.
            Last_Reduced_Matrix.append(Extract_subMatrix(ReductionN(M)[1]))        
            #print(Last_Reduced_Matrix)
    return Unitary_gates

Prints all the two level unitary gates.

In [13]:
def Unfold(Matrix):
    
    Gates = []
    i = 1
    for g in Reduce(Matrix):

        # If the matrix is of size NxN, print the matrix.
        if int(np.sqrt(g.size)) == N :
            
            Gates.append(["U"+str(i), g])
            #print('U'+str(i)+'\n', g,'\n\n')

        else:

            # If the matrix is of lower size other than N, expand the matrix into a NxN
            # matrix and print.
            
            Gates.append(["U"+str(i), Expand_Matrix(g, N) ])
            #print('U'+str(i)+'\n', Expand_Matrix(g, N),'\n\n')

        i += 1
        
    return Gates

In [16]:
K = Unfold(G)
F = np.identity(N, dtype=complex)
#for i in range(len(K)):
    #F = np.matmul(F,np.matrix(K[i][1]).getH())

In [19]:
len(K)

2016